In [4]:
# imports
import gffpandas.gffpandas as gffpd
from pybedtools import BedTool 
import pybedtools
import pandas as pd

In [5]:
def ann2df(file):
    '''convert annotations gff file to dataframe'''
    annotations = gffpd.read_gff3(file)
    sorted = annotations.df.sort_values(by="type")
    sorted['seq_id'] = sorted['seq_id'].str.replace('CHROMOSOME_', 'chr')
    return sorted

In [6]:
sort_by_type = ann2df('/Users/nutellachee/Downloads/c_elegans.canonical_bioproject.current.annotations.gff3')

/Users/nutellachee/anaconda3/lib/python3.11/site-packages/gffpandas/gffpandas.py:32: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',


In [7]:
sort_by_type.tail()

,seq_id,source,type,start,end,score,strand,phase,attributes
23684894,III,Allele,transposable_element_insertion_site,13055117,13055117,.,-,.,variation=WBVar00240129;public_name=pkP510;str...
35624054,V,NemaGENETAG_consortium,transposable_element_insertion_site,1927892,1927892,.,+,.,variation=WBVar00264011;public_name=ttTi41084;...
49151074,X,NemaGENETAG_consortium,transposable_element_insertion_site,9520583,9520583,.,+,.,variation=WBVar00261886;public_name=ttTi34859;...
14571986,II,NemaGENETAG_consortium,transposable_element_insertion_site,10944970,10944970,.,+,.,variation=WBVar00254572;public_name=ttTi4529;s...
28540411,IV,NemaGENETAG_consortium,transposable_element_insertion_site,7074863,7074863,.,+,.,variation=WBVar00258300;public_name=ttTi18910;...


In [8]:
sort_by_type['type'].unique()

array(['CDS', 'DNaseI_hypersensitive_site', 'G_quartet', 'PCR_product',
       'RNAi_reagent', 'SAGE_tag', 'SL1_acceptor_site',
       'SL2_acceptor_site', 'SNP', 'TF_binding_site', 'TSS_region',
       'antisense_RNA', 'assembly_component',
       'base_call_error_correction', 'binding_site', 'biological_region',
       'circular_ncRNA', 'complex_substitution', 'conserved_region',
       'deletion', 'duplication', 'enhancer', 'exon',
       'experimental_result_region', 'expressed_sequence_match',
       'five_prime_UTR', 'gene', 'histone_binding_site', 'insertion_site',
       'intron', 'inverted_repeat', 'lincRNA', 'low_complexity_region',
       'mRNA', 'mRNA_region', 'miRNA', 'miRNA_primary_transcript',
       'ncRNA', 'nc_primary_transcript', 'nucleotide_match', 'operon',
       'piRNA', 'point_mutation', 'polyA_signal_sequence', 'polyA_site',
       'polypeptide_motif', 'possible_base_call_error', 'pre_miRNA',
       'promoter', 'protein_coding_primary_transcript', 'protein_matc

In [10]:
sort_by_type['seq_id'].value_counts(dropna=False) 

seq_id
IV       10965377
V         9949289
II        9482996
X         8010681
I         7692388
III       7068678
MtDNA       55125
Name: count, dtype: int64

In [12]:
# rename each chromosome (seq_id) to match bed file in peaks
sort_by_type.loc[sort_by_type['seq_id'] == 'I', 'seq_id'] = 'chrI'
sort_by_type.loc[sort_by_type['seq_id'] == 'II', 'seq_id'] = 'chrII'
sort_by_type.loc[sort_by_type['seq_id'] == 'III', 'seq_id'] = 'chrIII'
sort_by_type.loc[sort_by_type['seq_id'] == 'IV', 'seq_id'] = 'chrIV'
sort_by_type.loc[sort_by_type['seq_id'] == 'V', 'seq_id'] = 'chrV'
sort_by_type.loc[sort_by_type['seq_id'] == 'MtDNA', 'seq_id'] = 'chrM'
sort_by_type.loc[sort_by_type['seq_id'] == 'X', 'seq_id'] = 'chrX'

# confirm renaming
sort_by_type['seq_id'].value_counts(dropna=False) 

seq_id
chrIV     10965377
chrV       9949289
chrII      9482996
chrX       8010681
chrI       7692388
chrIII     7068678
chrM         55125
Name: count, dtype: int64

In [16]:
# CONSTRUCT BED FILES FOR ce11 DATA
types = ['CDS', 'TSS_region', 'exon', 'five_prime_UTR', 'gene', 'intron', 'lincRNA', 'miRNA', 'ncRNA', 'promoter', 'three_prime_UTR']
def construct_beds(types):
    for type in types:
        df = sort_by_type[sort_by_type.type==type]
        df = df[['seq_id', 'start', 'end']]
        #df['seq_id'] = 'chr' + df['seq_id'].astype(str)
        output_bed = f'ce-11-beds/c-elegans-11-{type}.bed'
        df.to_csv(output_bed, sep='\t', header=False, index=False)

construct_beds(types)

In [ ]:
# GET INTRON BED
df = sort_by_type[sort_by_type.type=='intron']
intron_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-intron.bed'
#intron_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)

In [23]:
# GET EXON BED
df = sort_by_type[sort_by_type.type=='exon']
exon_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-exon.bed'
#exon_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)

In [24]:
d# GET CDS BED
f = sort_by_type[sort_by_type.type=='CDS']
cds_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-cds.bed'
#cds_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)

In [25]:
# GET GENE BED
df = sort_by_type[sort_by_type.type=='gene']
gene_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-genes.bed'
#gene_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)

In [16]:
# GET 5' UTR BED
df = sort_by_type[sort_by_type.type=='five_prime_UTR']
five_prime_utr_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-five-prime-utr.bed'
#five_prime_utr_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)

In [17]:
# GET 3' UTR BED
df = sort_by_type[sort_by_type.type=='three_prime_UTR']
three_prime_utr_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-three-prime-utr.bed'
#three_prime_utr_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)

In [18]:
# GET mRNA BED
df = sort_by_type[sort_by_type.type=='mRNA']
mrna_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-mrna.bed'
#mrna_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)

In [19]:
# GET ncRNA BED
df = sort_by_type[sort_by_type.type=='ncRNA']
ncrna_bed_df = df[['seq_id', 'start', 'end']]

output_bed = 'c-elegans-ncrna.bed'
#ncrna_bed_df.to_csv(output_bed, sep='\t', header=False, index=False)